In [ ]:
!pip install datasets torcheval fastprogress

In [ ]:
from datasets import load_dataset
import torchvision.transforms.functional as TF
import torch
from torch import nn,tensor
import torch.nn.functional as F
from operator import itemgetter
from torch.utils.data import default_collate, DataLoader

In [ ]:
# !pip install datasets

In [ ]:
g = 784
m = nn.Sequential(nn.Linear(g, 50), nn.ReLU(), nn.Linear(50, 10))
t = torch.randn(2, g)
m(t)

In [ ]:
x,y = 'image','label'
name = 'fashion_mnist'
dsr = load_dataset(name)

In [ ]:
def inplace(f):
  def _f(b):
    f(b)
    return b
  return _f

@inplace
def transform(b):
  b[x] = [torch.flatten(TF.to_tensor(i)) for i in b[x]]

dst = dsr.with_transform(transform)

In [ ]:
dst

In [ ]:
def get_dls(train, valid, batch_size, **kwargs):
  return (
      DataLoader(train, batch_size=batch_size, shuffle=True, **kwargs),
      DataLoader(valid, batch_size=batch_size*2, shuffle=False, **kwargs),
  )

def collate_dict(trds):
  g = itemgetter(*trds.features)
  def _f(b):
    return g(default_collate(b))
  return _f

class DataLoaders:
  def __init__(self, *ds):
    self.train,self.valid = ds[:2]

  @classmethod
  def from_dd(cls, ds, batch_size, **kwargs):
    f = collate_dict(ds['train'])
    return cls(*get_dls(*ds.values(), batch_size, collate_fn=f, **kwargs))

In [ ]:
t = [{'a': [1], 'b': [2]}, {'a': [1], 'b': [2]}]
k = default_collate(t)
g = itemgetter('a', 'b')
g(k)

In [ ]:
bs = 1024
dls = DataLoaders.from_dd(dst, batch_size=bs)
xb,yb = next(iter(dls.train))
xb.shape,yb.shape

In [ ]:
## Basic Learner

In [ ]:
import fastcore.all as fc
from torch import optim

In [ ]:
class Learner:
  def __init__(self, model, dls, lr, loss_func=F.cross_entropy, opt_func=optim.SGD):
    fc.store_attr()

  @property
  def training(self):
    return self.model.training

  def calc_stats(self):
    n = len(self.xb)
    self.accs.append((self.preds.argmax(dim=1)==self.yb).float().sum())
    self.losses.append(self.loss*n)
    self.ns.append(n)

  def one_batch(self):
    self.xb,self.yb = self.batch
    self.preds = self.model(self.xb)
    self.loss = self.loss_func(self.preds, self.yb)
    if self.training:
      self.loss.backward()
      self.opt.step()
      self.opt.zero_grad()
    self.calc_stats()

  def one_epoch(self, train):
    self.model.training = train
    self.dl = self.dls.train if train else self.dls.valid
    for self.batch in self.dl:
      self.one_batch()
    n = sum(self.ns)
    avgacc = sum(self.accs).item()/n
    avgloss = sum(self.losses).item()/n
    print(f'train:{train}, acc:{avgacc:.2}, loss:{avgloss:.2}')

  def fit(self, n_epochs):
    self.n_epochs = n_epochs
    self.ns,self.accs,self.losses = [],[],[]
    self.opt = self.opt_func(self.model.parameters(), lr=self.lr)
    for self.epoch in range(n_epochs):
      self.one_epoch(True)
      with torch.no_grad():
        self.one_epoch(False)

In [ ]:
m,nh,nout = 784,50,10
model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,nout))
learner = Learner(model, dls, lr=0.2)

In [ ]:
learner.fit(5)

In [ ]:
## Learner with Callbacks

In [ ]:
class CancelFitException(Exception):
  pass

class CancelEpochException(Exception):
  pass

class CancelBatchException(Exception):
  pass

In [ ]:
class Callback:
  order = 0

def run_cbs(cbs, method_nm, learn=None):
  for cb in sorted(cbs, key=lambda x: x.order):
    method = getattr(cb, method_nm, None)
    if method is not None:
      method(learn)

class CompletionCB(Callback):
  def before_epoch(self, learn):
    self.n = 0

  def after_batch(self, learn):
    self.n += 1

  def after_epoch(self, learn):
    print(f'total batch process: {self.n}')

cbs = [CompletionCB()]
run_cbs(cbs, 'before_epoch')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_epoch')

In [ ]:
### Metrics

In [ ]:
class Metric:
  def __init__(self, nm):
    self.nm = nm
    self.reset()

  def reset(self):
    self.ns,self.vals = [],[]

  def add(self, out, targ=None, n=1):
    val = self.calc(out, targ)
    self.ns.append(n)
    self.vals.append(val)

  def compute(self):
    ns = sum(self.ns) or 1
    avgval = sum(tensor(self.vals)*tensor(self.ns)).item()/ns
    print(f'{self.nm}: {avgval:.2}')

  def calc(self, out, targ=None):
    return out

In [ ]:
loss_metric = Metric('loss')
loss_metric.add(0.6, n=32)
loss_metric.add(0.9, n=2)
loss_metric.compute()

In [ ]:
class Accuracy(Metric):
  def calc(self, out, targ):
    return (out==targ).float().mean()

In [ ]:
acc_metric = Accuracy('accuracy')
acc_metric.add(tensor([0, 1, 2, 0, 1, 2]), tensor([0, 1, 1, 2, 1, 0]))
acc_metric.add(tensor([1, 1, 2, 0, 1]), tensor([0, 1, 1, 2, 1]))
acc_metric.compute()

In [ ]:
from torcheval.metrics import Mean, MulticlassAccuracy

In [ ]:
acc_metric_v2 = MulticlassAccuracy()
acc_metric_v2.update(tensor([0, 1, 2, 0, 1, 2]), tensor([0, 1, 1, 2, 1, 0]))
acc_metric_v2.update(tensor([1, 1, 2, 0, 1]), tensor([0, 1, 1, 2, 1]))
acc_metric_v2.compute()

In [ ]:
loss_metric_v2 = Mean()
loss_metric_v2.update(tensor(0.6), weight=32)
loss_metric_v2.update(tensor(0.9), weight=2)
loss_metric_v2.compute()

In [ ]:
type(Mean())

In [ ]:
from collections.abc import Mapping
from copy import copy

In [ ]:
# move all data to cpu before performing any metric calculation.
# this is to avoid any gradient accumulation + freeup gpu.
def to_cpu(x):
  if isinstance(x, Mapping):
    return {k:to_cpu(i) for k,i in x.items()}
  if isinstance(x, list):
    return [to_cpu(i) for i in x]
  if isinstance(x, tuple):
    return tuple(to_cpu(list(x)))
  res = x.detach().cpu()
  return res.float() if res.dtype==torch.float16 else res

In [ ]:
class MetricsCB(Callback):
  def __init__(self, *ms, **metrics):
    for i in ms:
      metrics[type(i).__name__] = i
    self.metrics = metrics
    self.all_metrics = copy(self.metrics)
    self.all_metrics['loss'] = self.loss = Mean()

  def before_fit(self, learn):
    learn.metrics = self

  def before_epoch(self, learn):
    [i.reset() for i  in self.all_metrics.values()]

  def after_batch(self, learn):
    x,y,*_ = to_cpu(learn.batch)
    for i in list(self.metrics.values()):
      i.update(to_cpu(learn.preds), y)
    self.loss.update(to_cpu(learn.loss), weight=len(x))

  def after_epoch(self, learn):
    log = {k:f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
    log['epoch'] = learn.epoch
    log['train'] = 'train' if learn.model.training else 'eval'
    self._log(log)

  def _log(self, d):
    print(d)

In [ ]:
t = {'a': [1]}
b = copy(t)
print(b)
print('-----')
b['a'].append(2)
print(b['a'])
print(t['a'])

In [ ]:
def get_model():
  return nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, nout))
get_model()

In [ ]:
## Flxible Learner

In [ ]:
class with_cbs:
  def __init__(self, nm):
    self.nm = nm

  def __call__(self, f):
    def _f(o, *args, **kwargs):
      try:
        o.callback(f'before_{self.nm}')
        f(o, *args, **kwargs)
        o.callback(f'after_{self.nm}')
      except globals()[f'Cancel{self.nm.title()}Exception']:
        pass
      finally:
        o.callback(f'cleanup_{self.nm}')
    return _f

In [ ]:
import torch.nn.functional as F
from torch import optim

In [ ]:
class Learner:
  def __init__(self, model, dls, lr=0.2, loss_func=F.cross_entropy, opt_func=optim.SGD, cbs=None):
     cbs = fc.L(cbs)
     fc.store_attr()

  def callback(self, method_nm):
    run_cbs(self.cbs, method_nm, self)

  @property
  def training(self):
    return self.model.training

  @with_cbs('batch')
  def _one_batch(self):
    self.xb,self.yb = self.batch
    self.preds = self.model(self.xb)
    self.loss = self.loss_func(self.preds, self.yb)
    if self.training:
      self.loss.backward()
      self.opt.step()
      self.opt.zero_grad()

  @with_cbs('epoch')
  def _one_epoch(self):
    for self.batch in self.dl:
      self._one_batch()

  def one_epoch(self, training):
    self.model.train(training)
    self.dl = self.dls.train if training else self.dls.valid
    self._one_epoch()

  @with_cbs('fit')
  def _fit(self, train, valid):
    for self.epoch in self.epochs:
      if train:
        self.one_epoch(True)
      if valid:
        self.one_epoch(False)

  def fit(self, n_epochs, lr=None, cbs=None, train=True, valid=True):
    cbs = fc.L(cbs)
    for cb in cbs:
      self.cbs.add(cb)
    lr = self.lr if lr is None else lr
    self.opt = self.opt_func(self.model.parameters(), lr=lr)
    self.n_epochs = n_epochs
    self.epochs = range(n_epochs)
    try:
      self._fit(train, valid)
    finally:
      for cb in cbs:
        self.cbs.remove(cb)

In [ ]:
model = get_model()
cbs = [MetricsCB(accuracy=MulticlassAccuracy())]
learner = Learner(model, dls=dls, cbs=cbs)
learner.fit(2)

In [ ]:
## Progress Bar

In [ ]:
from fastprogress import progress_bar, master_bar
import time

In [ ]:
t = master_bar(range(2))
for i in t:
  for j in progress_bar(range(2), parent=t, leave=False):
    print(j)
    time.sleep(1)

In [ ]:
mb = master_bar(range(1))
mb.names = ['cos', 'sin']
for i in mb:
  for k in progress_bar(range(1000), parent=mb):
    if k%300 ==0 or k+1==1000:
      pcnt = (1/1000)*k
      x1 = torch.linspace(0, pcnt*2*torch.pi, 10000)
      y1 = torch.cos(x1)
      y2 = torch.sin(x1)
      graphs = [[x1,y1], [x1, y2]]
      x_bounds,y_bounds = [0, 2*torch.pi], [-1, 1]
      mb.update_graph(graphs, x_bounds, y_bounds)

In [ ]:
class ProgressCB(Callback):
  order = MetricsCB.order+1

  def _log(self, d):
    if self.first:
      self.mbar.write(list(d.keys()), table=True)
      self.first = False
    self.mbar.write(list(d.values()), table=True)

  def before_fit(self, learn):
    self.losses = []
    self.val_losses = []
    self.first = True
    learn.epochs = self.mbar = master_bar(learn.epochs)
    if hasattr(learn, 'metrics'):
      learn.metrics._log = self._log

  def before_epoch(self, learn):
    learn.dl = progress_bar(learn.dl, parent=self.mbar, leave=False)

  def after_batch(self, learn):
    if learn.training:
      self.losses.append(learn.loss.item())
    else:
      self.val_losses.append(learn.loss.item())
    self.mbar.update_graph([[fc.L.range(self.losses), self.losses], [fc.L.range(self.val_losses), self.val_losses]])

  def after_epoch(self, learn):
    if not learn.training:
     self.mbar.update_graph([[fc.L.range(self.losses), self.losses], [fc.L.range(self.val_losses), self.val_losses]])


In [ ]:
model = get_model()
cbs = [MetricsCB(accuracy=MulticlassAccuracy()), ProgressCB()]
learner = Learner(model, dls, cbs=cbs)
learner.fit(2)

In [ ]:
## Learner with Custom Batch Operations

In [ ]:
from functools import partial

In [ ]:
def run_cbs(cbs, method_nm, learn=None):
  for cb in sorted(cbs, key=lambda x: x.order):
    method = getattr(cb, method_nm, None)
    if method is not None:
      method(learn)

In [ ]:
class Learner:
  def __init__(self, model, dls, lr=0.2, loss_func=F.cross_entropy, opt_func=optim.SGD, cbs=None):
    cbs = fc.L(cbs)
    fc.store_attr()

  def __getattr__(self, name):
    if name in ('predict', 'get_loss', 'backward', 'step', 'zero_grad'):
      return partial(self.callback, name)
    raise AttributeError(name)

  def callback(self, method_nm):
    run_cbs(self.cbs, method_nm, self)

  @property
  def training(self):
    return self.model.training

  @with_cbs('batch')
  def _one_batch(self):
    self.xb,self.yb = self.batch
    self.predict()
    self.callback('after_predict')
    self.get_loss()
    self.callback('after_loss')
    if self.training:
      self.backward()
      self.callback('after_backward')
      self.step()
      self.callback('after_step')
      self.zero_grad()

  @with_cbs('epoch')
  def _one_epoch(self):
    for self.batch in self.dl:
      self._one_batch()

  def one_epoch(self, train):
    self.model.train(train)
    self.dl = self.dls.train if train else self.dls.valid
    self._one_epoch()

  @with_cbs('fit')
  def _fit(self, train, valid):
    for self.epoch in self.epochs:
      if train:
        self.one_epoch(True)
      if valid:
        self.one_epoch(False)

  def fit(self, n_epochs, lr=None, cbs=None, train=True, valid=True):
    cbs = fc.L(cbs)
    lr = lr if lr else self.lr
    self.opt = self.opt_func(self.model.parameters(), lr=lr)
    self.n_epochs = n_epochs
    self.epochs = range(n_epochs)
    try:
       self._fit(train, valid)
    finally:
      for cb in cbs:
        self.cbs.remove(cb)


In [ ]:
class TrainCB(Callback):
  def predict(self, learn):
    learn.preds = learn.model(learn.xb)

  def get_loss(self, learn):
    learn.loss = learn.loss_func(learn.preds, learn.yb)

  def backward(self, learn):
    learn.loss.backward()

  def step(self, learn):
    learn.opt.step()

  def zero_grad(self, learn):
    learn.opt.zero_grad()


In [ ]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
cbs = [metrics, TrainCB(), ProgressCB()]
learner = Learner(model, dls, cbs=cbs)
learner.fit(1)

In [ ]:
class TrainLearner(Learner):
  def predict(self):
    self.preds = self.model(self.xb)

  def get_loss(self):
    self.loss = self.loss_func(self.preds, self.yb)

  def backward(self):
    self.loss.backward()

  def step(self):
    self.opt.step()

  def zero_grad(self):
    self.opt.zero_grad()

In [ ]:
class MomentumLearner(TrainLearner):
  def __init__(self, model, dls, lr=0.2, loss_func=F.cross_entropy, opt_func=optim.SGD, mom=0.85, cbs=None):
    self.mom = mom
    super().__init__(model, dls, lr=lr, loss_func=loss_func, opt_func=opt_func, cbs=cbs)

  def zero_grad(self):
    with torch.no_grad():
      for p in self.model.parameters():
        p.grad *= self.mom

In [ ]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
cbs = [metrics, ProgressCB()]
learner = MomentumLearner(model, dls, lr=0.1, cbs=cbs, mom=0.85)
learner.fit(5)

In [ ]:
learner = MomentumLearner(model, dls, lr=0.001, cbs=cbs, mom=0.85)
learner.fit(5)

In [ ]:
import math
import matplotlib.pyplot as plt

In [ ]:
class LRFinderCB(Callback):
  def __init__(self, lr_mult=1.3):
    fc.store_attr()

  def before_fit(self, learn):
    self.min = math.inf
    self.lrs,self.losses = [],[]

  def after_batch(self, learn):
    if not learn.training:
      raise CancelEpochException()
    self.lrs.append(learn.opt.param_groups[0]['lr'])
    loss = to_cpu(learn.loss)
    self.losses.append(loss)
    if loss < self.min:
      self.min = loss
    if loss > 3*self.min:
      raise CancelFitException()
    for g in learn.opt.param_groups:
      g['lr'] *= self.lr_mult

In [ ]:
lrfind = LRFinderCB()
cbs = [lrfind]
model = get_model()
learner = MomentumLearner(model, dls, cbs=cbs, lr=1e-4)
learner.fit(1)
plt.plot(lrfind.lrs, lrfind.losses)
plt.xscale('log')

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
class LRFinderCB(Callback):
  def __init__(self, gamma=1.3, max_mult=3):
    fc.store_attr()

  def before_fit(self, learn):
    self.sched = ExponentialLR(learn.opt, gamma=self.gamma)
    self.min = math.inf
    self.losses,self.lrs = [],[]

  def after_batch(self, learn):
    if not learn.training:
      raise CancelEpochException()
    loss = to_cpu(learn.loss)
    self.losses.append(loss)
    self.lrs.append(learn.opt.param_groups[0]['lr'])
    if loss < self.min:
      self.min = loss
    if math.isnan(loss) or loss>self.max_mult*self.min:
      raise CancelFitException()
    self.sched.step()

  def cleanup_fit(self, learn):
    plt.plot(self.lrs, self.losses)
    plt.xscale('log')

In [ ]:
model = get_model()
cbs = [LRFinderCB()]
learner = MomentumLearner(model, dls, lr=1e-4, cbs=cbs)
learner.fit(1)

In [ ]:
for i in range(5):
  try:
    print('first', i)
    try:
      # raise CancelEpochException()
      raise CancelFitException()
      print('inside one')
    except CancelEpochException:
      print('epoch exception')
  except CancelFitException:
    print('fit exception')
    break